In [119]:
import tensorflow as tf
import pandas as pd

X = pd.read_csv('data/boston_preprocessing_X.csv')
y = pd.read_csv('data/boston_preprocessing_y.csv')

train_num = 300
valid_num = 100
test_num = len(X) - train_num -valid_num

X_train = X[:train_num]
y_train = y[:train_num]

X_valid = X[train_num:train_num+valid_num]
y_valid = y[train_num:train_num+valid_num]

X_test = X[train_num+valid_num:]
y_test = y[train_num+valid_num:]

X_train.shape, X_valid.shape, X_test.shape

((300, 12), (100, 12), (106, 12))

In [120]:
X_train = tf.cast(X_train, dtype=tf.float32)
y_train = tf.cast(y_train, dtype=tf.float32)
X_valid = tf.cast(X_valid, dtype=tf.float32)
y_valid = tf.cast(y_valid, dtype=tf.float32)
X_test = tf.cast(X_test, dtype=tf.float32)
y_test = tf.cast(y_test, dtype=tf.float32)

In [121]:
# W = tf.Variable(tf.random.normal([12, 1], mean=0.0, stddev=1.0, dtype=tf.float32))
W = tf.Variable(tf.zeros([12, 1]), dtype=tf.float32)
B = tf.Variable(tf.zeros(1), dtype=tf.float32)
W, B

(<tf.Variable 'Variable:0' shape=(12, 1) dtype=float32, numpy=
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>)

In [122]:
def model(x, w, b):
    return tf.matmul(x, w) + b


def loss(x, y, w, b):
    err = model(x, w, b) - y
    squared_err = tf.square(err)
    return tf.reduce_mean(squared_err)


def grad(x, y, w, b):
    with tf.GradientTape() as tape:
        loss_ = loss(x, y, w, b)
    return tape.gradient(loss_, [w, b])

In [123]:
import numpy as np

training_epochs = 4000
learning_rate = 1e-3
batch_size = 5
optimizer = tf.keras.optimizers.SGD(learning_rate)


loss_list_train = []
loss_list_valid = []
best_loss_test = np.inf
total_step = int(train_num/batch_size)

for epoch in range(training_epochs):
    for step in range(total_step):
        xs = X_train[step*batch_size:(step+1)*batch_size, :]
        ys = y_train[step*batch_size:(step+1)*batch_size, :]

        grads = grad(xs, ys, W, B)
        optimizer.apply_gradients(zip(grads, [W, B]))

    loss_train = loss(X_train, y_train, W, B).numpy()
    loss_valid = loss(X_valid, y_valid, W, B).numpy()
    loss_list_train.append(loss_train)
    loss_list_valid.append(loss_valid)
    if loss_valid <= np.min(loss_list_valid):
        best_loss_test = loss(X_test, y_test, W, B).numpy()
    print("epoch={:4d}, train_loss={:.4f}, valid_loss={:.4f}".format(epoch+1, loss_train, loss_valid))
print("Best model in test_loss: ", best_loss_test)

epoch=   1, train_loss=376.1162, valid_loss=541.0463
epoch=   2, train_loss=224.4110, valid_loss=630.8448
epoch=   3, train_loss=148.5627, valid_loss=703.0731
epoch=   4, train_loss=105.5245, valid_loss=748.7456
epoch=   5, train_loss=78.8802, valid_loss=774.5723
epoch=   6, train_loss=61.4772, valid_loss=787.8022
epoch=   7, train_loss=49.7257, valid_loss=793.5076
epoch=   8, train_loss=41.6033, valid_loss=794.8139
epoch=   9, train_loss=35.8789, valid_loss=793.5485
epoch=  10, train_loss=31.7687, valid_loss=790.7584
epoch=  11, train_loss=28.7603, valid_loss=787.0409
epoch=  12, train_loss=26.5134, valid_loss=782.7422
epoch=  13, train_loss=24.7993, valid_loss=778.0673
epoch=  14, train_loss=23.4632, valid_loss=773.1423
epoch=  15, train_loss=22.3990, valid_loss=768.0494
epoch=  16, train_loss=21.5334, valid_loss=762.8448
epoch=  17, train_loss=20.8152, valid_loss=757.5698
epoch=  18, train_loss=20.2084, valid_loss=752.2553
epoch=  19, train_loss=19.6874, valid_loss=746.9258
epoch=  

In [124]:
# test_house_id = np.random.randint(0, test_num)
# y = y_test[test_house_id]
# y_pred = model(X_test, W, B)[test_house_id]
# y_predict = tf.reshape(y_pred, ()).numpy()
# print("House id", test_house_id, "Actual value", *y.numpy(), "Predicted value", y_predict)